In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *


spark = SparkSession.builder.getOrCreate()


df = spark.read.csv("./datasets/Titanic.csv", header=True, inferSchema=True)

df = df.withColumn("age", df["age"].cast(IntegerType()))
df = df.withColumn("fare", df["fare"].cast(FloatType()))


df.createOrReplaceTempView("titanic")


children_count = spark.sql("SELECT COUNT(name) FROM titanic WHERE age < 18")


children_count.show()

+-----------+
|count(name)|
+-----------+
|        154|
+-----------+



In [37]:
fare_and_ages = spark.sql("SELECT fare, COLLECT_LIST(DISTINCT(age)) as ages "
                          "from titanic "
                          "GROUP BY fare "
                          "ORDER BY fare ASC ")
fare_and_ages.show()

+------+--------------------+
|  fare|                ages|
+------+--------------------+
|  NULL|                [60]|
|   0.0|[39, 38, 19, 36, ...|
|3.1708|                 [9]|
|4.0125|                [20]|
|   5.0|                [33]|
|6.2375|                [61]|
|6.4375|                [34]|
|  6.45|                [43]|
|6.4958|        [18, 21, 34]|
|  6.75|            [19, 18]|
|6.8583|                  []|
|  6.95|                [30]|
| 6.975|            [27, 45]|
|   7.0|                [47]|
|7.0458|                [29]|
|  7.05|[25, 38, 24, 23, ...|
|7.0542|            [51, 17]|
| 7.125|    [17, 22, 35, 41]|
|7.1417|                [24]|
| 7.225|[26, 25, 30, 15, ...|
+------+--------------------+
only showing top 20 rows



In [27]:
max_ages = spark.sql("""
    SELECT MAX(size(ages)) as max_len 
    FROM (SELECT COLLECT_LIST(age) as ages FROM titanic GROUP BY fare)
""").collect()[0][0]

max_ages

58

In [36]:
columns = ', '.join([f"ages[{i}] as age_{i+1}" for i in range(max_ages)])

fare_and_ages = spark.sql(f"""
    SELECT fare, {columns}
    FROM (
        SELECT fare, COLLECT_LIST(DISTINCT(age)) as ages 
        FROM titanic 
        GROUP BY fare
        ORDER BY fare ASC
    )
""")
fare_and_ages.show(truncate=False)

+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
|fare  |age_1|age_2|age_3|age_4|age_5|age_6|age_7|age_8|age_9|age_10|age_11|age_12|age_13|age_14|age_15|age_16|age_17|age_18|age_19|age_20|age_21|age_22|age_23|age_24|age_25|age_26|age_27|age_28|age_29|age_30|age_31|age_32|age_33|age_34|age_35|age_36|age_37|age_38|age_39|age_40|age_41|age_42|age_43|age_44|age_45|age_46|age_47|age_48|age_49|age_50|age_51|age_52|age_53|age_54|age_55|age_56|age_57|age_58|
+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+

In [31]:
survival_status = spark.sql(f'''
    SELECT survived, Residence, AVG(age) 
    FROM titanic
    GROUP BY survived, Residence
    ORDER BY survived, Residence
''')

survival_status.show(truncate=False)

+--------+---------+------------------+
|survived|Residence|avg(age)          |
+--------+---------+------------------+
|0       |0        |40.77319587628866 |
|0       |1        |30.957055214723926|
|0       |2        |27.54874651810585 |
|1       |0        |35.044444444444444|
|1       |1        |27.305882352941175|
|1       |2        |25.545893719806763|
+--------+---------+------------------+



In [35]:
age_diff = spark.sql("""
    SELECT t.*, 
           (max_age_per_class.max_age - t.age) AS age_difference
    FROM titanic t
    JOIN (
        SELECT pclass, MAX(age) as max_age
        FROM titanic
        GROUP BY pclass
    ) max_age_per_class
    ON t.pclass = max_age_per_class.pclass
""")

age_diff.show()

age_diff.write.csv("Titanic_Age_Difference.csv", header=True)

+------+--------+---------+--------------------+---+-----+-----+------------------+-------+-----+--------+----+----+--------------------+------+--------------+
|pclass|survived|Residence|                name|age|sibsp|parch|            ticket|   fare|cabin|embarked|boat|body|           home.dest|Gender|age_difference|
+------+--------+---------+--------------------+---+-----+-----+------------------+-------+-----+--------+----+----+--------------------+------+--------------+
|     3|       0|        0| Abbing, Mr. Anthony| 42|    0|    0|         C.A. 5547|   7.55|     |       S|    |    |                    |     0|            32|
|     3|       0|        0|Abbott, Master. E...| 13|    0|    2|         C.A. 2673|  20.25|     |       S|    |    | East Providence, RI|     0|            61|
|     3|       0|        0|Abbott, Mr. Rossm...| 16|    1|    1|         C.A. 2673|  20.25|     |       S|    | 190| East Providence, RI|     0|            58|
|     3|       1|        0|Abbott, Mrs. 